In [2]:
# Calcula a perda de percurso usando o modelo de log-distância.
def channel_path_loss(PL_CO_lin, d, alpha):
    """
    Calcula a perda de percurso usando o modelo de log-distância.

    Parâmetros:
    PL_CO_lin : float
        Perda de percurso na distância de referência (em escala linear).
    d : float ou array
        Distância entre transmissor e receptor.
    alpha : float
        Expoente de perda de percurso.

    Retorna:
    PL : float ou array
        Perda de percurso na distância d.
    """
    PL = PL_CO_lin * (d ** -alpha)
    return PL

In [3]:
# Modela o canal de comunicação Rician.
import numpy as np

def channel_model_H(Hlos, Hnlos, kappa, N_realizacao):
    """
    Modela o canal de comunicação para cada dispositivo.

    Parâmetros:
    Hlos         -> Lista de matrizes (N_realizacao x K x N) representando os componentes LoS
    Hnlos        -> Lista de matrizes (N_realizacao x K x N) representando os componentes NLoS
    kappa        -> Fator LoS do modelo de fading quase-estático de Rician (escalar)
    N_realizacao -> Número de realizações do canal (escalar)

    Retorna:
    H -> Lista de matrizes (N_realizacao x K x N) representando a matriz de canal resultante
    """
    H = []

    for r in range(N_realizacao):
        if kappa == 0:
            H.append(Hnlos[r])  # Apenas o componente NLoS
        else:
            H.append(
                np.sqrt(1 / (1 + kappa)) * Hnlos[r] +
                np.sqrt(kappa / (1 + kappa)) * Hlos[r]
            )  # Modelo Rician

    return H


In [4]:
# Modelo de canal NLoS baseado na distribuição de Rayleigh.
import numpy as np
from scipy.stats import rayleigh

def channel_model_nlos(M, N, kappa, N_realizacao):
    """
    Modelo de canal NLoS baseado na distribuição de Rayleigh.

    Parâmetros:
    M - Número de antenas transmissoras na PB
    N - Número de antenas receptoras
    kappa - Fator LOS do modelo de fading quasi-estático de Rician
    N_realizacao - Número de realizações do canal

    Retorno:
    Hnlos - Lista de matrizes do canal NLoS
    """
    Hnlos = []
    pdf1 = rayleigh(scale=1)  # Distribuição de Rayleigh com parâmetro 'b' = 1

    for _ in range(N_realizacao):
        H_real = pdf1.rvs(size=(M, N))  # Parte real do canal
        H_imag = pdf1.rvs(size=(M, N))  # Parte imaginária do canal
        Hnlos.append(H_real + 1j * H_imag)  # Construção do canal complexo

    return Hnlos


In [5]:
#  Modelo de canal LoS ?
import numpy as np

def channel_model_hlos(M, N, d, kappa, N_realizacao):
    """
    Modela o canal Line-of-Sight (LoS) para comunicação sem fio.

    Parâmetros:
    M (int): Número de antenas transmissoras na PB.
    N (int): Número de antenas receptoras.
    d (float): Distância entre transmissor e receptor.
    kappa (float): Fator de LoS do modelo de fading Rician.
    N_realizacao (int): Número de realizações do canal.

    Retorna:
    Hlos (list of np.ndarray): Lista de matrizes do canal LoS para cada realização.
    """
    ang = np.arctan(d)  # Posição angular do dispositivo EH (não está sendo usada diretamente)

    Hlos = []
    for _ in range(N_realizacao):
        Hlos.append(np.random.randn(M, N) + 1j * np.random.randn(M, N))

    return Hlos


In [63]:
import numpy as np
import math

# Definir parâmetros
seed = 1
np.random.seed(seed)  # Garante reprodutibilidade

# Parâmetros da Rede Sem Fio
N_Realizacao = 2
N = 10  # Número de antenas
M = 20  # Número de elementos no STAR-RIS
kappa = 3.0  # Fator de Rician
R_min = 2
Sigma = -80  # dBm
Sigma_lin = (10**(Sigma/10)) / (10**3)
Pt = 1000 * (10**-3)  # Potência total de transmissão

# Variáveis de Perda de Caminho (Path Loss)
PL_D0 = 1  # m - Distância de referência
PL_C0 = -30  # dB - PL na distância de referência
PL_C0_lin = 10 ** (PL_C0 / 10)  # Conversão de dB para valor linear
dG = 70  # Distância entre a BS e o STAR-RIS
dv = 2   # Distância vertical
dt = 20  # Distância horizontal entre o STAR-RIS e o UE no lado de transmissão
dr = 50  # Distância horizontal entre o STAR-RIS e o UE no lado de reflexão

# Modelo de Canal
# BS -> STAR-RIS (G) - Canal LoS
Alpha_BS = 2  # Expoente de perda de percurso (PL)

# BS -> UEr (hd) - Canal LoS
Alpha_BS_UEr = 2  # Expoente de perda de percurso (PL)

# STAR-RIS -> UE r (hr) - Canal com desvanecimento de Rayleigh
Alpha_SUr = 2  # Expoente de perda de percurso (PL)

# STAR-RIS -> UE t (ht) - Canal com desvanecimento de Rayleigh
Alpha_SUt = 2  # Expoente de perda de percurso (PL)


# Distâncias
dBr = np.sqrt((dG - dr)**2 + dv**2)  # BS-UEr
dBt = np.sqrt((dG + dt)**2 + dv**2)  # BS-UEt
dSt = np.sqrt(dt**2 + dv**2)  # STAR-RIS - UEt
dSr = np.sqrt(dr**2 + dv**2)  # STAR-RIS - UEr

# Parâmetros do PSO
Nit = 1000
c1 = 0.7
c2 = 0.7
wmin = 0.2
wmax = 0.9
TamPop = 20

# Main
# Análise de Perda de Caminho (Path Loss)

# BS para STAR-RIS
PL_BS = channel_path_loss(PL_C0_lin, dG, Alpha_BS)

# BS para UEr
PL_BS_UEr = channel_path_loss(PL_C0_lin, dBr, Alpha_BS_UEr)

# STAR-RIS para UEr
PL_SUr = channel_path_loss(PL_C0_lin, dSr, Alpha_SUr)

# STAR-RIS para UEt
PL_SUt = channel_path_loss(PL_C0_lin, dSt, Alpha_SUt)

#print(f"PL_BS: {PL_BS}, PL_BS_UEr: {PL_BS_UEr}, PL_SUr: {PL_SUr}, PL_SUt: {PL_SUt}")
Best = np.zeros((2,1000), dtype=np.float64)
for real in range(1, N_Realizacao + 1):
    print(f'OMA - Realizacao: {real}')

    # --> Channel Definition

    # BS to STAR-RIS
    hG_los = channel_model_hlos(M, N, dG, kappa, 1)
    hG_nlos = channel_model_nlos(M, N, kappa, 1)
    hG = channel_model_H(hG_los, hG_nlos, kappa, 1)
    hG[0] = math.sqrt(PL_BS) * hG[0]

    # BS to UEr
    hd_los = channel_model_hlos(1, N, dBr, kappa, 1)
    hd_nlos = channel_model_nlos(1, N, kappa, 1)
    hd = channel_model_H(hd_los, hd_nlos, kappa, 1)
    hd[0] = math.sqrt(PL_BS_UEr) * hd[0]

    # STAR-RIS to UEr
    hr_los = channel_model_hlos(M, 1, dSr, kappa, 1)
    hr_nlos = channel_model_nlos(M, 1, kappa, 1)
    hr = channel_model_H(hr_los, hr_nlos, kappa, 1)
    hr[0] = math.sqrt(PL_SUr) * hr[0]

    # STAR-RIS to UEt
    ht_los = channel_model_hlos(M, 1, dSt, kappa, 1)
    ht_nlos = channel_model_nlos(M, 1, kappa, 1)
    ht = channel_model_H(ht_los, ht_nlos, kappa, 1)
    ht[0] = math.sqrt(PL_SUt) * ht[0]

    #Best = np.zeros((2,5), dtype=np.float64)
    for g in range(1, Nit + 1):
        if g == 1:
          # First Swarm
          # Position
          Wr_pos = np.exp(-1j * 2 * np.pi / np.random.randint(1, 101, (N, TamPop)))
          Wt_pos = np.exp(-1j * 2 * np.pi / np.random.randint(1, 101, (N, TamPop)))

          Wr_pos = (np.sqrt(Pt) * Wr_pos) / (np.sqrt(N) * np.abs(Wr_pos))
          Wt_pos = (np.sqrt(Pt) * Wt_pos) / (np.sqrt(N) * np.abs(Wt_pos))

          Amp_Tr_pos = np.random.rand(M, TamPop)
          Amp_Tt_pos = 1 - Amp_Tr_pos

          Theta_Tr_pos = 2 * np.pi / np.random.randint(1, 101, (M, TamPop))
          Theta_Tt_pos = 2 * np.pi / np.random.randint(1, 101, (M, TamPop))

          Tr_pos = Amp_Tr_pos * np.exp(-1j * Theta_Tr_pos)
          Tt_pos = Amp_Tt_pos * np.exp(-1j * Theta_Tt_pos)

          # Velocity
          Wr_vel = np.exp(-1j * 2 * np.pi / np.random.randint(1, 1001, (N, TamPop)))
          Wt_vel = np.exp(-1j * 2 * np.pi / np.random.randint(1, 1001, (N, TamPop)))

          Amp_Tr_vel = 0.5 * np.random.rand(M, TamPop)
          Amp_Tt_vel = 0.5 * np.random.rand(M, TamPop)

          Theta_Tr_vel = 2 * np.pi / np.random.randint(1, 101, (M, TamPop))
          Theta_Tt_vel = 2 * np.pi / np.random.randint(1, 101, (M, TamPop))

          Tr_vel = Amp_Tr_pos * np.exp(-1j * Theta_Tr_vel)
          Tt_vel = Amp_Tt_pos * np.exp(-1j * Theta_Tt_vel)

          power_pos = np.zeros((2, TamPop))
          power_pos[0, :] = np.random.rand(1, TamPop)
          power_pos[1, :] = 1 - power_pos[0, :]

          power_vel = np.random.rand(2, TamPop)

          SNRr = np.zeros(TamPop, dtype=np.float64)
          SNRt = np.zeros(TamPop, dtype=np.float64)
          Rt = np.zeros(TamPop, dtype=np.float64)
          Rr = np.zeros(TamPop, dtype=np.float64)
          Fitness = np.zeros(TamPop, dtype=np.float64)
          Fitness0 = np.zeros(TamPop, dtype=np.float64)
          for pop in range(TamPop):
              # Calcule o produto das matrizes
              part1_r = np.dot(np.dot(hr[0].conj().T, np.diag(Tr_pos[:, pop])), hG[0]) + hd[0]
              part2_r = np.dot(part1_r, Wr_pos[:, pop])

              part1_t = np.dot(np.dot(ht[0].conj().T, np.diag(Tt_pos[:, pop])), hG[0])
              part2_t = np.dot(part1_t, Wt_pos[:, pop])

              # Garantir que estamos lidando com um valor escalar
              SNRr[pop] = np.abs(np.squeeze(part2_r))**2  # Squeeze para garantir que é um escalar
              SNRt[pop] = np.abs(np.squeeze(part2_t))**2  # Squeeze para garantir que é um escalar
              #print(SNRr[pop])

              Rt[pop] = 0.5 * np.log2(1 + (Pt * SNRt[pop] / Sigma_lin))
              Rr[pop] = 0.5 * np.log2(1 + (Pt * SNRr[pop] / Sigma_lin))
              if Rr[pop] >= R_min and Rt[pop] >= R_min:
                Fitness[pop] = Rr[pop] + Rt[pop]
              else:
                if Rr[pop] < R_min:
                  if Rt[pop] < R_min:
                    Fitness[pop] = (Rr[pop] - R_min) + (Rt[pop] - R_min)
                  else:
                    Fitness[pop] = (Rr[pop] - R_min)
                else:
                  Fitness[pop] = Rt[pop] - R_min  # 1000 * abs(SINRt[pop] - SNR_lin)

              # Define Pbest and Gbest
              # Pbest
              Wr_pos_pbest = Wr_pos;
              Wt_pos_pbest = Wt_pos;

              power_pos_pbest = power_pos;

              Theta_Tr_pos_pbest = Theta_Tr_pos;
              Theta_Tt_pos_pbest = Theta_Tt_pos;

              Amp_Tr_pos_pbest = Amp_Tr_pos;
              Amp_Tt_pos_pbest = Amp_Tt_pos;

              Fitness_pbest = Fitness;

              # Gbest
              Fitness_gbest = np.max(Fitness)
              pos_gbest = np.argmax(Fitness)

              Wr_pos_gbest = Wr_pos[:, pos_gbest]
              Wt_pos_gbest = Wt_pos[:, pos_gbest]

              power_pos_gbest = power_pos[:, pos_gbest]

              Amp_Tr_pos_gbest = Amp_Tr_pos[:, pos_gbest]
              Amp_Tt_pos_gbest = Amp_Tt_pos[:, pos_gbest]

              Theta_Tr_pos_gbest = Theta_Tr_pos[:, pos_gbest]
              Theta_Tt_pos_gbest = Theta_Tt_pos[:, pos_gbest]

              # Update the Velocity and Position
              w = wmax - ((wmax - wmin) * g / Nit)

              for pop in range(TamPop):
                Wr_vel[:, pop] = w * Wr_vel[:, pop] + c1 * np.random.rand() * (Wr_pos_pbest[:, pop] - Wr_pos[:, pop]) + c2 * np.random.rand() * (Wr_pos_gbest - Wr_pos[:, pop])
                Wt_vel[:, pop] = w * Wt_vel[:, pop] + c1 * np.random.rand() * (Wt_pos_pbest[:, pop] - Wt_pos[:, pop]) + c2 * np.random.rand() * (Wt_pos_gbest - Wt_pos[:, pop])

                Amp_Tr_vel[:, pop] = w * Amp_Tr_vel[:, pop] + c1 * np.random.rand() * (Amp_Tr_pos_pbest[:, pop] - Amp_Tr_pos[:, pop]) + c2 * np.random.rand() * (Amp_Tr_pos_gbest - Amp_Tr_pos[:, pop])
                Amp_Tt_vel[:, pop] = w * Amp_Tt_vel[:, pop] + c1 * np.random.rand() * (Amp_Tt_pos_pbest[:, pop] - Amp_Tt_pos[:, pop]) + c2 * np.random.rand() * (Amp_Tt_pos_gbest - Amp_Tt_pos[:, pop])

                Theta_Tr_vel[:, pop] = w * Theta_Tr_vel[:, pop] + c1 * np.random.rand() * (Theta_Tr_pos_pbest[:, pop] - Theta_Tr_pos[:, pop]) + c2 * np.random.rand() * (Theta_Tr_pos_gbest - Theta_Tr_pos[:, pop])
                Theta_Tt_vel[:, pop] = w * Theta_Tt_vel[:, pop] + c1 * np.random.rand() * (Theta_Tt_pos_pbest[:, pop] - Theta_Tt_pos[:, pop]) + c2 * np.random.rand() * (Theta_Tt_pos_gbest - Theta_Tt_pos[:, pop])

                power_vel[:, pop] = w * power_vel[:, pop] + c1 * np.random.rand() * (power_pos_pbest[:, pop] - power_pos[:, pop]) + c2 * np.random.rand() * (power_pos_gbest - power_pos[:, pop])

                Theta_Tr_pos[:, pop] = Theta_Tr_pos[:, pop] + Theta_Tr_vel[:, pop]
                Theta_Tt_pos[:, pop] = Theta_Tt_pos[:, pop] + Theta_Tt_vel[:, pop]

                Amp_Tr_pos[:, pop] = np.abs(Amp_Tr_pos[:, pop] + Amp_Tr_vel[:, pop])
                Amp_Tt_pos[:, pop] = np.abs(Amp_Tt_pos[:, pop] + Amp_Tt_vel[:, pop])

                power_pos[:, pop] = power_pos[:, pop] + power_vel[:, pop]


                if power_pos[0, pop] >= 1:
                    power_pos[0, pop] = np.random.rand()
                    power_pos[1, pop] = 1 - power_pos[0, pop]
                else:
                    power_pos[1, pop] = 1 - power_pos[0, pop]

                if power_pos[0, pop] < 0:
                    power_pos[0, pop] = np.random.rand()
                    power_pos[1, pop] = 1 - power_pos[0, pop]
                else:
                    power_pos[1, pop] = 1 - power_pos[0, pop]

                for m in range(M):
                    if Amp_Tr_pos[m, pop] < 1:
                        Amp_Tt_pos[m, pop] = 1 - Amp_Tr_pos[m, pop]
                    else:
                        Amp_Tt_pos[m, pop] = np.random.rand()
                        Amp_Tr_pos[m, pop] = 1 - Amp_Tt_pos[m, pop]

                Wr_pos[:, pop] = Wr_pos[:, pop] + Wr_vel[:, pop]
                Wt_pos[:, pop] = Wt_pos[:, pop] + Wt_vel[:, pop]

              Wr_pos = (np.sqrt(Pt) * Wr_pos) / (np.sqrt(N) * np.abs(Wr_pos))
              Wt_pos = (np.sqrt(Pt) * Wt_pos) / (np.sqrt(N) * np.abs(Wt_pos))

              Tr_pos = Amp_Tr_pos * np.exp(1j * Theta_Tr_pos)
              Tt_pos = Amp_Tt_pos * np.exp(1j * Theta_Tt_pos)

              # Loop para cálculo da Fitness
              for pop in range(TamPop):
                  # Cálculo de SNRr e SNRt
                  #SNRr[pop] = np.abs((np.dot(hr[0].T, np.diag(Tr_pos[:, pop])) @ hG[0] + hd[0]) @ Wr_pos[:, pop])**2
                  #SNRt[pop] = np.abs((np.dot(ht[0].T, np.diag(Tt_pos[:, pop])) @ hG[0]) @ Wt_pos[:, pop])**2

                  # Calcule o produto das matrizes
                  part1_r = np.dot(np.dot(hr[0].conj().T, np.diag(Tr_pos[:, pop])), hG[0]) + hd[0]
                  part2_r = np.dot(part1_r, Wr_pos[:, pop])

                  part1_t = np.dot(np.dot(ht[0].conj().T, np.diag(Tt_pos[:, pop])), hG[0])
                  part2_t = np.dot(part1_t, Wt_pos[:, pop])


                  # Cálculo de SNRr e SNRt
                  SNRr[pop] = np.abs(np.squeeze(part2_r))**2  # Squeeze para garantir que é um escalar
                  SNRt[pop] = np.abs(np.squeeze(part2_t))**2  # Squeeze para garantir que é um escalar

                  # Cálculo de Rt e Rr
                  Rt[pop] = 0.5 * np.log2(1 + (Pt * SNRt[pop] / Sigma_lin))
                  Rr[pop] = 0.5 * np.log2(1 + (Pt * SNRr[pop] / Sigma_lin))

                  # Verificação e cálculo de Fitness
                  if Rr[pop] >= R_min and Rt[pop] >= R_min:
                      Fitness[pop] = Rr[pop] + Rt[pop]
                  else:
                      if Rr[pop] < R_min:
                          if Rt[pop] < R_min:
                              Fitness[pop] = (Rr[pop] - R_min) + (Rt[pop] - R_min)
                          else:
                              Fitness[pop] = (Rr[pop] - R_min)
                      else:
                          Fitness[pop] = Rt[pop] - R_min  # 1000 * np.abs(SINRt[pop] - SNR_lin)
          #print(f"{Fitness_gbest} g={g}")
          #Best = np.zeros((real, g), dtype=np.float64)
          Best[real-1, g-1] = Fitness_gbest
          #print(Best)
        else:
          # Define Pbest and Gbest

          # Pbest
          for pop in range(TamPop):
              if Fitness_pbest[pop] < Fitness[pop]:
                  Wr_pos_pbest[:, pop] = Wr_pos[:, pop]
                  Wt_pos_pbest[:, pop] = Wt_pos[:, pop]

                  power_pos_pbest[:, pop] = power_pos[:, pop]

                  Theta_Tr_pos_pbest[:, pop] = Theta_Tr_pos[:, pop]
                  Theta_Tt_pos_pbest[:, pop] = Theta_Tt_pos[:, pop]

                  Amp_Tr_pos_pbest[:, pop] = Amp_Tr_pos[:, pop]
                  Amp_Tt_pos_pbest[:, pop] = Amp_Tt_pos[:, pop]

                  Fitness_pbest[pop] = Fitness[pop]

          # Gbest
          if Fitness_gbest < np.max(Fitness):
              Fitness_gbest = np.max(Fitness)
              pos_gbest = np.argmax(Fitness)

              Wr_pos_gbest = Wr_pos[:, pos_gbest]
              Wt_pos_gbest = Wt_pos[:, pos_gbest]

              power_pos_gbest = power_pos[:, pos_gbest]

              Amp_Tr_pos_gbest = Amp_Tr_pos[:, pos_gbest]
              Amp_Tt_pos_gbest = Amp_Tt_pos[:, pos_gbest]

              Theta_Tr_pos_gbest = Theta_Tr_pos[:, pos_gbest]
              Theta_Tt_pos_gbest = Theta_Tt_pos[:, pos_gbest]

          # Atualiza a inércia (w)
          w = wmax - ((wmax - wmin) * g / Nit)

          for pop in range(TamPop):
              Wr_vel[:, pop] = w * Wr_vel[:, pop] + c1 * np.random.rand() * (Wr_pos_pbest[:, pop] - Wr_pos[:, pop]) + c2 * np.random.rand() * (Wr_pos_gbest - Wr_pos[:, pop])
              Wt_vel[:, pop] = w * Wt_vel[:, pop] + c1 * np.random.rand() * (Wt_pos_pbest[:, pop] - Wt_pos[:, pop]) + c2 * np.random.rand() * (Wt_pos_gbest - Wt_pos[:, pop])

              Amp_Tr_vel[:, pop] = w * Amp_Tr_vel[:, pop] + c1 * np.random.rand() * (Amp_Tr_pos_pbest[:, pop] - Amp_Tr_pos[:, pop]) + c2 * np.random.rand() * (Amp_Tr_pos_gbest - Amp_Tr_pos[:, pop])
              Amp_Tt_vel[:, pop] = w * Amp_Tt_vel[:, pop] + c1 * np.random.rand() * (Amp_Tt_pos_pbest[:, pop] - Amp_Tt_pos[:, pop]) + c2 * np.random.rand() * (Amp_Tt_pos_gbest - Amp_Tt_pos[:, pop])

              Theta_Tr_vel[:, pop] = w * Theta_Tr_vel[:, pop] + c1 * np.random.rand() * (Theta_Tr_pos_pbest[:, pop] - Theta_Tr_pos[:, pop]) + c2 * np.random.rand() * (Theta_Tr_pos_gbest - Theta_Tr_pos[:, pop])
              Theta_Tt_vel[:, pop] = w * Theta_Tt_vel[:, pop] + c1 * np.random.rand() * (Theta_Tt_pos_pbest[:, pop] - Theta_Tt_pos[:, pop]) + c2 * np.random.rand() * (Theta_Tt_pos_gbest - Theta_Tt_pos[:, pop])

              power_vel[:, pop] = w * power_vel[:, pop] + c1 * np.random.rand() * (power_pos_pbest[:, pop] - power_pos[:, pop]) + c2 * np.random.rand() * (power_pos_gbest - power_pos[:, pop])

              Theta_Tr_pos[:, pop] += Theta_Tr_vel[:, pop]
              Theta_Tt_pos[:, pop] += Theta_Tt_vel[:, pop]

              Amp_Tr_pos[:, pop] = np.abs(Amp_Tr_pos[:, pop] + Amp_Tr_vel[:, pop])
              Amp_Tt_pos[:, pop] = np.abs(Amp_Tt_pos[:, pop] + Amp_Tt_vel[:, pop])

              power_pos[:, pop] += power_vel[:, pop]

              if power_pos[0, pop] >= 1:
                  power_pos[0, pop] = np.random.rand()
                  power_pos[1, pop] = 1 - power_pos[0, pop]
              else:
                  power_pos[1, pop] = 1 - power_pos[0, pop]

              if power_pos[0, pop] < 0:
                  power_pos[0, pop] = np.random.rand()
                  power_pos[1, pop] = 1 - power_pos[0, pop]
              else:
                  power_pos[1, pop] = 1 - power_pos[0, pop]

              for m in range(M):
                  if Amp_Tr_pos[m, pop] < 1:
                      Amp_Tt_pos[m, pop] = 1 - Amp_Tr_pos[m, pop]
                  else:
                      Amp_Tt_pos[m, pop] = np.random.rand()
                      Amp_Tr_pos[m, pop] = 1 - Amp_Tt_pos[m, pop]

              Wr_pos[:, pop] += Wr_vel[:, pop]
              Wt_pos[:, pop] += Wt_vel[:, pop]

              Wr_pos = (np.sqrt(Pt) * Wr_pos) / (np.sqrt(N) * np.abs(Wr_pos))
              Wt_pos = (np.sqrt(Pt) * Wt_pos) / (np.sqrt(N) * np.abs(Wt_pos))

              Tr_pos = Amp_Tr_pos * np.exp(1j * Theta_Tr_pos)
              Tt_pos = Amp_Tt_pos * np.exp(1j * Theta_Tt_pos)

              for pop in range(TamPop):
                  #SNRr[pop] = np.abs((hr[0].T @ np.diag(Tr_pos[:, pop]) @ hG[0] + hd[0]) @ Wr_pos[:, pop])**2
                  #SNRt[pop] = np.abs((ht[0].T @ np.diag(Tt_pos[:, pop]) @ hG[0]) @ Wt_pos[:, pop])**2

                  # Calcule o produto das matrizes
                  part1_r = np.dot(np.dot(hr[0].conj().T, np.diag(Tr_pos[:, pop])), hG[0]) + hd[0]
                  part2_r = np.dot(part1_r, Wr_pos[:, pop])

                  part1_t = np.dot(np.dot(ht[0].conj().T, np.diag(Tt_pos[:, pop])), hG[0])
                  part2_t = np.dot(part1_t, Wt_pos[:, pop])

                  # Cálculo de SNRr e SNRt
                  SNRr[pop] = np.abs(np.squeeze(part2_r))**2  # Squeeze para garantir que é um escalar
                  SNRt[pop] = np.abs(np.squeeze(part2_t))**2  # Squeeze para garantir que é um escalar

                  Rt[pop] = 0.5 * np.log2(1 + (Pt * SNRt[pop] / Sigma_lin))
                  Rr[pop] = 0.5 * np.log2(1 + (Pt * SNRr[pop] / Sigma_lin))

                  if Rr[pop] >= R_min and Rt[pop] >= R_min:
                      Fitness[pop] = Rr[pop] + Rt[pop]
                  else:
                      if Rr[pop] < R_min:
                          if Rt[pop] < R_min:
                              Fitness[pop] = (Rr[pop] - R_min) + (Rt[pop] - R_min)
                          else:
                              Fitness[pop] = (Rr[pop] - R_min)
                      else:
                          Fitness[pop] = Rt[pop] - R_min  # 1000 * np.abs(SINRt[pop] - SNR_lin)

        #print(f"{g}: {Fitness_gbest}: [{real-1},{g-1}]")
        #Best = np.zeros((real, g), dtype=np.float64)
        #Best = np.zeros((2,1000), dtype=np.float64)
        #print(Fitness_gbest)
        if Fitness_gbest != 0:
          Best[real-1, g-1] = Fitness_gbest
        else:
          Best[real-1, g-1] = 0


#print(Fitness_gbest)
print(Best)
print(np.mean(Best))
r, c = Best.shape  # Obtém o número de linhas e colunas
if r == 1 and c > 1:
    print("Best é um vetor linha.")
elif r > 1 and c == 1:
    print("Best é um vetor coluna.")
elif r > 1 and c > 1:
    print(f"Best é uma matriz. [{r}][{c}]")
else:
    print("Best é um escalar.")



OMA - Realizacao: 1
OMA - Realizacao: 2
[[12.12415269 12.12415269 12.12415269 ... 12.41389011 12.41389011
  12.41389011]
 [12.97386024 13.20320434 13.20320434 ... 13.81954084 13.81954084
  13.81954084]]
13.108506234723594
Best é uma matriz. [2][1000]
